# **DIABETES PREDICTION PROJECT**

### IMPORT LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, precision_recall_curve
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
import shap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Enhanced styling options
plt.rcParams['figure.figsize'] = (12, 8)  # Set default figure size for better detail
plt.rcParams['font.size'] = 12  # Increase base font size for better readability
plt.rcParams['axes.titlesize'] = 16  # Make titles more prominent
plt.rcParams['axes.labelsize'] = 14  # Make axis labels stand out

# For medical-specific visualizations
sns.set_context("talk")  # Larger elements, suited for presentations
plt.rcParams['axes.titleweight'] = 'bold'  # Bold titles for emphasis

In [ ]:
# 1. DATA ACQUISITION
print("Phase 1: Data Acquisition and Initial Exploration")
print("-------------------------------------------------")

#Load the datasets


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv')

In [ ]:
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
df.columns = column_names

In [ ]:
print(f"Dataset shape: {df.shape}")
print("\nFirst few records:")
print(df.head())


# **Exploratory Data Analysis (EDA)**

In [ ]:
df.info()

In [ ]:
summary = df.describe()
print("Summary Statistics:")

In [ ]:
zero_counts = (df == 0).sum()  # Count zeros in each column
print(zero_counts)

In [ ]:
print("\n Converting impossible zero values to missing values (NaN):")

In [ ]:
zero_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

In [ ]:
medical_reasons = {
    'Glucose': 'Blood glucose cannot be zero in living patients',
    'BloodPressure': 'Blood pressure of zero would indicate death',
    'SkinThickness': 'Skin fold thickness cannot be zero',
    'Insulin': 'Missing insulin test rather than actual zero',
    'BMI': 'BMI cannot be zero for a person with physical mass'
}
for col in zero_columns:
    df[col] = df[col].replace(0, np.nan)

In [ ]:
print(df.isna().sum())

In [ ]:
print("\nTarget Distribution:")
outcome_counts = df['Outcome'].value_counts()
print(outcome_counts)
print(f"Percentage of diabetic patients: {outcome_counts[1] / len(df) * 100:.2f}%")


In [ ]:
#Visualization by the outcome
plt.figure(figsize=(15, 10))
for i, col in enumerate(df.columns[:-1]):
    plt.subplot(3, 3, i+1)
    sns.histplot(data=df, x=col, hue='Outcome', element='step', kde=True, bins=20)
    plt.title(f'Distribution of {col} by Outcome', fontsize=10)
plt.tight_layout()
plt.savefig('feature_distributions.png')

In [ ]:
# Correlation analysis
plt.figure(figsize=(10,8))
correlation_matrix = df.corr().round(2)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.savefig('correlation_matrix.png')


# **Data Preprocessing**

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")



In [ ]:
preprocessing_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),  # Advanced imputation
    ('scaler', RobustScaler())  # Robust to outliers
])

In [ ]:
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

In [ ]:
X_train_processed_df = pd.DataFrame(
    X_train_processed,
    columns=X_train.columns
)
print(X_train_processed_df.describe().round(2))

In [ ]:
df.head()

# **Feature Engineering**

In [ ]:
X_train_processed_df['Glucose_BMI'] = X_train_processed_df['Glucose'] * X_train_processed_df['BMI']
X_train_processed_df['Age_BMI'] = X_train_processed_df['Age'] * X_train_processed_df['BMI']
X_train_processed_df['Glucose_Age'] = X_train_processed_df['Glucose'] * X_train_processed_df['Age']


In [ ]:
X_test_processed_df = pd.DataFrame(
    X_test_processed,
    columns=X_train.columns
)
X_test_processed_df['Glucose_BMI'] = X_test_processed_df['Glucose'] * X_test_processed_df['BMI']
X_test_processed_df['Age_BMI'] = X_test_processed_df['Age'] * X_test_processed_df['BMI']
X_test_processed_df['Glucose_Age'] = X_test_processed_df['Glucose'] * X_test_processed_df['Age']

print(f"Features after engineering: {X_train_processed_df.columns.tolist()}")
print(f"New feature set shape: {X_train_processed_df.shape}")

# Convert back to numpy arrays for model training
X_train_final = X_train_processed_df.values
X_test_final = X_test_processed_df.values


# **Model Selection and Training**


In [ ]:
initial_model = RandomForestClassifier(random_state=42)

In [ ]:
# Example of data preprocessing steps before Model Selection and Training

# Assuming you have already completed data preprocessing steps like handling missing values,
# feature engineering, scaling, etc.

# Let's assume we have the processed data as `X_train_processed_df` and `y_train`

from sklearn.model_selection import train_test_split

# Example of splitting the data (use your processed data here)
X = X_train_processed_df  # Features after preprocessing and engineering
y = y_train  # Target variable (diabetes diagnosis)

# Split data into training and testing sets
X_train_final, X_test_final, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, you can proceed with the model selection and training phase

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score

print("\nPhase 5: Model Selection and Training")
print("----------------------------------")

# 1. Initial Random Forest Model
initial_model = RandomForestClassifier(random_state=42)

# 2. Cross-validation strategy using StratifiedKFold to ensure class distribution is maintained
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation evaluation: ROC-AUC is used as it is a suitable metric for medical classification problems
cv_scores = cross_val_score(initial_model, X_train_final, y_train, cv=cv_strategy, scoring='roc_auc')

# Display cross-validation results
print(f"Initial model cross-validation ROC-AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# 3. Hyperparameter tuning using GridSearchCV
# This will search over a grid of hyperparameters and evaluate using ROC-AUC
param_grid = {
    'n_estimators': [100, 200, 300],            # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],             # Depth of trees
    'min_samples_split': [2, 5, 10],             # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],               # Minimum samples required to be at a leaf node
    'max_features': ['sqrt', 'log2', None]      # Number of features to consider at each split
}

# GridSearchCV for hyperparameter optimization with ROC-AUC as the scoring metric
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=cv_strategy,                             # Cross-validation strategy
    scoring='roc_auc',                          # Scoring based on ROC-AUC
    n_jobs=-1,                                  # Use all CPU cores to speed up the search
    verbose=1                                   # Display progress while searching
)

# 4. Fit the grid search to the data to find the best hyperparameters
grid_search.fit(X_train_final, y_train)

# 5. Display the best hyperparameters and the best score from cross-validation
print(f"\nBest hyperparameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# 6. Train the final model with the best hyperparameters
best_model = grid_search.best_estimator_
best_model.fit(X_train_final, y_train)

# 7. Predict on the test data and evaluate the model
y_pred = best_model.predict(X_test_final)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"\nTest ROC-AUC: {roc_auc:.4f}")


# **Model Evaluation**

In [ ]:
# Make predictions
y_pred = best_model.predict(X_test_final)
y_pred_proba = best_model.predict_proba(X_test_final)[:, 1]

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


In [ ]:
# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.savefig('confusion_matrix.png')

In [ ]:
print(classification_report(y_test, y_pred))


In [ ]:
# ROC Curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'RandomForest (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.savefig('roc_curve.png')

In [ ]:
# Precision-Recall Curve
plt.figure(figsize=(8, 6))
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
plt.plot(recall_curve, precision_curve)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.savefig('precision_recall_curve.png')

# **Model Interpretation**

In [ ]:
# Understanding the model's decision-making is crucial for clinical applications
feature_names = list(X_train_processed_df.columns)
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Feature Importance')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=90)
plt.tight_layout()
plt.savefig('feature_importance.png')

In [ ]:
for i in range(len(importances)):
    print(f"{i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

In [ ]:
# Permutation Importance - more reliable for correlated features
perm_importance = permutation_importance(
    best_model, X_test_final, y_test, n_repeats=10, random_state=42
)

plt.figure(figsize=(10, 6))
sorted_idx = perm_importance.importances_mean.argsort()
plt.boxplot(perm_importance.importances[sorted_idx].T,
            vert=False, labels=[feature_names[i] for i in sorted_idx])
plt.title("Permutation Importances")
plt.tight_layout()
plt.savefig('permutation_importance.png')


In [ ]:

# Ensure feature names are available, usually you can take it from the dataframe columns
feature_names = X_test_final.columns

# SHAP Explainer for Random Forest
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test_final)

# If it's a binary classification, shap_values will have two sets, one for each class.
# For binary classification, use shap_values[1] for the positive class.
# For multi-class classification, loop through all classes, or use shap_values[0] for the first class.

# Check if it's binary or multi-class and adjust accordingly
if isinstance(shap_values, list):
    shap_values_class = shap_values[1]  # For the positive class in binary classification
else:
    shap_values_class = shap_values

# Plotting SHAP summary plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values_class, X_test_final, feature_names=feature_names, show=False)
plt.title("SHAP Feature Impact Summary")
plt.tight_layout()
plt.savefig('shap_summary.png')


# **MODEL DEPLOYMENT PREPARATION**

In [ ]:
# Save the preprocessing pipeline and model for deployment
import joblib

# Save the preprocessing pipeline
joblib.dump(preprocessing_pipeline, 'diabetes_preprocessing_pipeline.pkl')

# Save the feature engineering function
def engineer_features(df):
    """
    Add engineered features to the input dataframe
    """
    df_copy = df.copy()
    df_copy['Glucose_BMI'] = df_copy['Glucose'] * df_copy['BMI']
    df_copy['Age_BMI'] = df_copy['Age'] * df_copy['BMI']
    df_copy['Glucose_Age'] = df_copy['Glucose'] * df_copy['Age']
    return df_copy

# Save the model
joblib.dump(best_model, 'diabetes_prediction_model.pkl')


In [ ]:
df.columns

In [ ]:
# Create a sample prediction function
def predict_diabetes_risk(patient_data):
    """
    Make diabetes risk prediction for new patient data

    Parameters:
    patient_data (dict): Dictionary with patient features

    Returns:
    dict: Prediction results including probability and risk category
    """
    # Convert input to DataFrame
    patient_df = pd.DataFrame([patient_data])

    # Preprocess the data
    patient_processed = preprocessing_pipeline.transform(patient_df)
    patient_processed_df = pd.DataFrame(
        patient_processed,
        columns=patient_df.columns
    )

    # Apply feature engineering
    patient_final = engineer_features(patient_processed_df)

    # Make prediction
    risk_prob = best_model.predict_proba(patient_final.values)[0, 1]
    risk_prediction = 1 if risk_prob >= 0.5 else 0

    # Risk category
    if risk_prob < 0.2:
        risk_category = "Low Risk"
    elif risk_prob < 0.5:
        risk_category = "Moderate Risk"
    elif risk_prob < 0.7:
        risk_category = "High Risk"
    else:
        risk_category = "Very High Risk"

    return {
        "prediction": risk_prediction,
        "probability": risk_prob,
        "risk_category": risk_category
    }


In [ ]:
sample_patient = pd.DataFrame({
    'Pregnancies': [2],
    'Glucose': [120],
    'BloodPressure': [70],
    'SkinThickness': [20],
    'Insulin': [79],
    'BMI': [25.5],
    'DiabetesPedigreeFunction': [0.5],
    'Age': [32]
})

# Get training data columns and their order
training_columns = X_train_processed_df.columns

# Ensure all features exist in the sample patient data, and in the correct order
for col in training_columns:
    if col not in sample_patient.columns:
        sample_patient[col] = 0  # or suitable default

# Reorder columns to match training data
sample_patient = sample_patient[training_columns]

# Now predict
risk_prob = best_model.predict_proba(sample_patient)[0, 1]
risk_prediction = 1 if risk_prob >= 0.5 else 0

print(f" Predicted Diabetes Risk Probability: {risk_prob:.2f}")
print(f" Predicted Diabetes Risk Class (0=No, 1=Yes): {risk_prediction}")